# 🎯 Valentina Facial LoRA Trainer - Google Colab Edition

Este notebook treina uma **LoRA de identidade facial** para a Valentina usando FLUX.1-dev + dataset otimizado no Google Colab.

## 🎯 Foco: Identidade Visual (NÃO NSFW)
- **Objetivo**: Gerar a Valentina com máxima consistência facial
- **Dataset**: Imagens da pasta `valentina_identity_4lora_dataset_flux`
- **Características**: 25 anos, rosto oval, olhos amendoados, sem tatuagens
- **Trigger Word**: `vltna woman`

## 📋 Stack Otimizada para Identidade:
- **Base**: FLUX.1-dev (máxima qualidade facial)
- **Dataset**: 18 imagens com seeds sequenciais (máxima consistência)
- **Parâmetros**: Conservadores para preservar características faciais
- **Output**: LoRA facial da Valentina para uso local com mflux

⚠️ **Importante**: Execute as células em ordem e aguarde a conclusão de cada etapa.

In [1]:
# --- Configurações Principais ---
BASE_MODEL_ID = "black-forest-labs/FLUX.1-dev" # Modelo base oficial para identidade
DATASET_ZIP_FILENAME = "valentina_identity_4lora_dataset_flux.zip" # Dataset de identidade gerado

# --- Caminhos no Ambiente Colab ---
COLAB_MODELS_PATH = "/content/models"
COLAB_DATASET_PATH = "/content/dataset"
COLAB_OUTPUT_PATH = "/content/output_lora"
COLAB_LOGS_PATH = "/content/logs"

# --- Parâmetros de Treinamento da LoRA de Identidade ---
INSTANCE_PROMPT = "a photo of vltna woman" # Token único para Valentina
CLASS_PROMPT = "a photo of a woman" # Prompt de classe para regularização

# Configurações de resolução e qualidade (baseado no dataset gerado)
RESOLUTION = 1024  # Mesma resolução do dataset
CENTER_CROP = True
RANDOM_FLIP = False # NUNCA flipar para preservar características faciais

# Batch sizes otimizados para identidade (conservadores)
TRAIN_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 4 # Batch efetivo menor para preservar detalhes

# Learning rates CONSERVADORES para preservar identidade facial
LEARNING_RATE = 5e-5 # Reduzido para preservar características faciais
UNET_LR = 5e-5
TEXT_ENCODER_LR = 3e-6 # Muito menor para text encoder

# Scheduler e warmup otimizados para identidade
LR_SCHEDULER = "cosine_with_restarts"
LR_WARMUP_STEPS = 50 # Reduzido para dataset menor
LR_NUM_CYCLES = 1

# Steps de treinamento (calculado para dataset de identidade: ~80-120 steps por imagem)
MAX_TRAIN_STEPS = 1500 # Para 18 imagens = ~83 steps/imagem (conservador)
SAVE_STEPS = 300
VALIDATION_EPOCHS = 3 # Reduzido para dataset menor

# Arquitetura LoRA otimizada para IDENTIDADE FACIAL
LORA_RANK = 64 # Rank menor para preservar identidade (era 128)
LORA_ALPHA = 32 # Alpha = rank/2 para estabilidade
LORA_DROPOUT = 0.05 # Dropout menor para melhor aprendizado

# Configurações de precisão e otimização
MIXED_PRECISION = "bf16" # Melhor qualidade se suportado
USE_8BIT_ADAM = True
ADAM_BETA1 = 0.9
ADAM_BETA2 = 0.999
ADAM_WEIGHT_DECAY = 0.01
ADAM_EPSILON = 1e-8
MAX_GRAD_NORM = 1.0

# Memory optimization
GRADIENT_CHECKPOINTING = True
ENABLE_XFORMERS = True
USE_CPU_OFFLOAD = False # Manter na GPU para velocidade

# Regularização para IDENTIDADE FACIAL
PRIOR_LOSS_WEIGHT = 1.0
SNR_GAMMA = 5.0 # Para melhor qualidade com ruído

# Checkpointing conservador
CHECKPOINTING_STEPS = 300
CHECKPOINTS_TOTAL_LIMIT = 3
RESUME_FROM_CHECKPOINT = None

# Seeds para reprodutibilidade (alinhado com dataset)
SEED = 42 # Mesmo seed base do dataset

# Configurações específicas para IDENTIDADE (não NSFW)
USE_MIDJOURNEY_LORA = False # Desabilitado - foco em identidade pura
VALIDATION_PROMPT = "a photo of vltna woman, professional portrait photography" # Prompt de validação para identidade

print("⚙️ Configurações otimizadas para IDENTIDADE FACIAL carregadas:")
print(f"📊 Steps totais: {MAX_TRAIN_STEPS}")
print(f"🎯 Batch efetivo: {TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"🧠 LoRA Rank: {LORA_RANK} (reduzido para preservar identidade)")
print(f"🎨 Base Model: {BASE_MODEL_ID}")
print(f"💼 Dataset: {DATASET_ZIP_FILENAME}")
print(f"🎭 Trigger: '{INSTANCE_PROMPT}'")
print(f"🔬 Foco: IDENTIDADE FACIAL (não NSFW)")

⚙️ Configurações otimizadas para IDENTIDADE FACIAL carregadas:
📊 Steps totais: 1500
🎯 Batch efetivo: 4
🧠 LoRA Rank: 64 (reduzido para preservar identidade)
🎨 Base Model: black-forest-labs/FLUX.1-dev
💼 Dataset: valentina_identity_4lora_dataset_flux.zip
🎭 Trigger: 'a photo of vltna woman'
🔬 Foco: IDENTIDADE FACIAL (não NSFW)


## Célula 2: Setup do Ambiente Colab (Dependências)

In [2]:
# 📦 INSTALAÇÃO OTIMIZADA DE DEPENDÊNCIAS VIA UV
print("🔧 Instalando dependências usando uv para maior robustez...")

# 1. Instalar uv
print("⚙️ Instalando uv...")
!pip install -q uv
print("✅ uv instalado.")

# 2. Definir o conteúdo do requirements.txt baseado no pyproject.toml do mflux
requirements_content = """\
torch>=2.1.0,<2.4.0
torchvision>=0.16.0,<0.19.0
torchaudio>=2.1.0,<2.4.0
diffusers[torch]>=0.27.0,<1.0
transformers>=4.44.0,<5.0
accelerate>=0.32.0,<1.0
safetensors>=0.4.0,<0.5.0
xformers>=0.0.27,<0.0.29
pillow>=10.0.0,<11.0.0
opencv-python>=4.10.0,<5.0
huggingface-hub>=0.24.5,<1.0
sentencepiece>=0.2.0,<0.3.0
tokenizers>=0.19.0,<0.20.0
protobuf>=5.27.0,<6.0.0
numpy>=2.0.0,<3.0.0
requests>=2.32.0,<3.0.0
scipy>=1.14.0,<2.0.0
matplotlib>=3.9.0,<4.0.0
omegaconf>=2.3.0,<3.0.0
einops>=0.8.0,<0.9.0
invisible-watermark>=0.2.0,<0.3.0
compel>=2.0.0,<3.0.0
wandb>=0.17.0,<0.18.0
peft>=0.12.0,<0.13.0
bitsandbytes>=0.43.0,<0.44.0
gradio>=4.39.0,<5.0.0
albumentations>=1.4.0,<2.0.0
imageio>=2.34.0,<3.0.0
scikit-image>=0.24.0,<0.25.0
tqdm>=4.66.0,<5.0.0
ftfy>=6.2.0,<7.0.0
tensorboard>=2.16.0,<3.0.0
easydict>=1.13.0,<2.0.0
clean-fid==0.1.35
torchmetrics>=1.4.0,<2.0.0
kornia>=0.7.0,<0.8.0
lpips>=0.1.4,<0.2.0
controlnet_aux>=0.0.7,<0.0.8
segment-anything>=1.0.0,<2.0.0
rembg[gpu]>=2.0.56,<3.0.0
moviepy>=1.0.3,<2.0.0
typer>=0.12.0,<0.13.0
rich>=13.7.0,<14.0.0
shellingham>=1.5.0,<2.0.0
"""

# 3. Criar o arquivo requirements.txt no ambiente do Colab
requirements_file_path = "/content/valentina_flux_requirements.txt"
with open(requirements_file_path, "w") as f:
    f.write(requirements_content)
print(f"📄 {requirements_file_path} criado com dependências do mflux.")

# 4. Instalar dependências usando uv pip install
print(f"🚀 Instalando dependências de {requirements_file_path} com uv...")
print("🕒 Isso pode levar alguns minutos...")
!uv pip install -q -r {requirements_file_path} --extra-index-url https://download.pytorch.org/whl/cu121 --index-strategy unsafe-best-match

print("✅ Dependências instaladas com sucesso usando uv!")
print("🎯 Ambiente otimizado para treinamento de LoRA FLUX")

# Verificação final de CUDA e PyTorch
import torch
print(f"🔥 PyTorch: {torch.__version__}")
print(f"🎮 CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 VRAM total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
    print(f"🧹 VRAM livre: {torch.cuda.memory_allocated() / 1024**3:.1f}GB")
else:
    print("⚠️ CUDA não detectado - usando CPU (MUITO LENTO)")

# Limpeza de memória inicial
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("✅ Verificação de ambiente concluída!")

🔧 Instalando dependências usando uv para maior robustez...
⚙️ Instalando uv...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 120.4 MB/s eta 0:00:00
✅ uv instalado.
📄 /content/valentina_flux_requirements.txt criado com dependências do mflux.
🚀 Instalando dependências de /content/valentina_flux_requirements.txt com uv...
🕒 Isso pode levar alguns minutos...
✅ Dependências instaladas com sucesso usando uv!
🎯 Ambiente otimizado para treinamento de LoRA FLUX
🔥 PyTorch: 2.3.1+cu121
🎮 CUDA disponível: True
🎯 GPU: NVIDIA A100-SXM4-40GB
💾 VRAM total: 39.6GB
🧹 VRAM livre: 0.0GB
✅ Verificação de ambiente concluída!


In [3]:
# 🖥️ Verificação e Otimização da GPU
import torch
import subprocess

print("🔍 Verificando configuração da GPU...")
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader,nounits

# Detectar capacidades da GPU
gpu_name = subprocess.check_output(["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"]).decode().strip()
print(f"📱 GPU detectada: {gpu_name}")

# Ajustar configurações baseado na GPU
if "T4" in gpu_name:
    print("🔧 Otimizações para Tesla T4")
    MIXED_PRECISION = "fp16"  # T4 funciona melhor com fp16
    TRAIN_BATCH_SIZE = 1
    GRADIENT_ACCUMULATION_STEPS = 6
elif "V100" in gpu_name:
    print("🔧 Otimizações para V100")
    MIXED_PRECISION = "fp16"
    TRAIN_BATCH_SIZE = 1
    GRADIENT_ACCUMULATION_STEPS = 8
elif "A100" in gpu_name or "H100" in gpu_name:
    print("🔧 Otimizações para GPU high-end")
    MIXED_PRECISION = "bf16"  # Melhor qualidade
    TRAIN_BATCH_SIZE = 2
    GRADIENT_ACCUMULATION_STEPS = 4
else:
    print("🔧 Configurações padrão")
    MIXED_PRECISION = "fp16"

print(f"✅ Configurações ajustadas: {MIXED_PRECISION}, batch={TRAIN_BATCH_SIZE}")

# Limpar cache da GPU
torch.cuda.empty_cache()
print(f"🧹 Cache da GPU limpo. Memória livre: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

🔍 Verificando configuração da GPU...
NVIDIA A100-SXM4-40GB, 40960, 40501
📱 GPU detectada: NVIDIA A100-SXM4-40GB
🔧 Otimizações para GPU high-end
✅ Configurações ajustadas: bf16, batch=2
🧹 Cache da GPU limpo. Memória livre: 42.5GB


In [4]:
from huggingface_hub import notebook_login

print("Por favor, faça login na sua conta Hugging Face para baixar os modelos.")
notebook_login()

Por favor, faça login na sua conta Hugging Face para baixar os modelos.


## Célula 4: Criação da Estrutura de Diretórios no Colab

In [5]:
# 📁 Criação da Estrutura de Diretórios
import os
from pathlib import Path

# Criar todos os diretórios necessários
directories = [
    COLAB_MODELS_PATH,
    COLAB_DATASET_PATH,
    COLAB_OUTPUT_PATH,
    COLAB_LOGS_PATH,
    f"{COLAB_DATASET_PATH}/instance_images",
    f"{COLAB_DATASET_PATH}/class_images",
    f"{COLAB_OUTPUT_PATH}/checkpoints",
    f"{COLAB_OUTPUT_PATH}/samples"
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"📂 Criado: {directory}")

print("\n✅ Estrutura de diretórios criada com sucesso!")

📂 Criado: /content/models
📂 Criado: /content/dataset
📂 Criado: /content/output_lora
📂 Criado: /content/logs
📂 Criado: /content/dataset/instance_images
📂 Criado: /content/dataset/class_images
📂 Criado: /content/output_lora/checkpoints
📂 Criado: /content/output_lora/samples

✅ Estrutura de diretórios criada com sucesso!


## Célula 5: Upload e Preparação do Dataset de Identidade

In [6]:
# 📤 Upload e Preparação do Dataset de Identidade
from google.colab import files
import zipfile
import shutil
from PIL import Image
import os

print("📥 === UPLOAD DO DATASET DE IDENTIDADE ===")
print(f"Por favor, faça upload do arquivo: {DATASET_ZIP_FILENAME}")
print("🧬 Dataset gerado pelo valentina_dataset_generator_colab.ipynb")
print("📊 Contém 18 imagens com seeds sequenciais para máxima consistência facial")
uploaded_dataset = files.upload()

if DATASET_ZIP_FILENAME in uploaded_dataset:
    print(f"📦 Extraindo {DATASET_ZIP_FILENAME}...")
    with zipfile.ZipFile(DATASET_ZIP_FILENAME, 'r') as zip_ref:
        zip_ref.extractall(COLAB_DATASET_PATH)

    # Encontrar as imagens extraídas
    instance_images_path = f"{COLAB_DATASET_PATH}/instance_images"

    # Verificar estrutura do dataset
    extracted_files = []
    for root, dirs, files in os.walk(COLAB_DATASET_PATH):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                extracted_files.append(os.path.join(root, file))

    print(f"🖼️ Encontradas {len(extracted_files)} imagens de identidade")

    # Mover imagens para pasta de instância se necessário
    if not os.path.exists(instance_images_path):
        os.makedirs(instance_images_path, exist_ok=True)

    processed_count = 0
    for i, img_path in enumerate(extracted_files):
        new_path = f"{instance_images_path}/valentina_{i:03d}.png"

        # Verificar se já não está na pasta correta
        if os.path.dirname(img_path) == instance_images_path:
            continue

        # Converter e redimensionar se necessário
        with Image.open(img_path) as img:
            # Converter para RGB se necessário
            if img.mode != 'RGB':
                img = img.convert('RGB')

            # Verificar se já está no tamanho correto
            if img.size == (RESOLUTION, RESOLUTION):
                # Salvar direto se já está no tamanho certo
                img.save(new_path, 'PNG', quality=95)
            else:
                # Redimensionar mantendo aspect ratio
                img.thumbnail((RESOLUTION, RESOLUTION), Image.Resampling.LANCZOS)

                # Criar imagem quadrada com padding
                new_img = Image.new('RGB', (RESOLUTION, RESOLUTION), (255, 255, 255))
                paste_x = (RESOLUTION - img.width) // 2
                paste_y = (RESOLUTION - img.height) // 2
                new_img.paste(img, (paste_x, paste_y))

                new_img.save(new_path, 'PNG', quality=95)

        processed_count += 1
        print(f"✅ Processada: {os.path.basename(img_path)} -> {os.path.basename(new_path)}")

    print(f"\n📊 Dataset de identidade preparado: {len(extracted_files)} imagens")
    print(f"🎯 Imagens processadas: {processed_count}")
    !ls -la {instance_images_path}

    # Verificar se existe metadata do dataset
    metadata_path = f"{COLAB_DATASET_PATH}/dataset_metadata.json"
    if os.path.exists(metadata_path):
        print(f"📋 Metadata do dataset encontrado: {metadata_path}")
        with open(metadata_path, 'r') as f:
            import json
            metadata = json.load(f)
            if isinstance(metadata, list) and len(metadata) > 0:
                first_meta = metadata[0]
                print(f"🧬 Configuração do dataset:")
                print(f"   • Arquitetura: {first_meta.get('architecture', 'N/A')}")
                print(f"   • Foco: {first_meta.get('optimization_focus', 'N/A')}")
                print(f"   • Base Model: {first_meta.get('base_model', 'N/A')}")
                print(f"   • Seeds: {metadata[0].get('seed', 'N/A')} a {metadata[-1].get('seed', 'N/A')}")
else:
    print(f"❌ ERRO: {DATASET_ZIP_FILENAME} não encontrado!")
    print("🔍 Certifique-se de que:")
    print("1. Gerou o dataset usando valentina_dataset_generator_colab.ipynb")
    print("2. Baixou o arquivo valentina_identity_4lora_dataset_flux.zip")
    print("3. Está fazendo upload do arquivo correto")
    raise FileNotFoundError("Dataset de identidade não foi enviado")

print("\n✅ Dataset de identidade preparado com sucesso!")
print("🧬 Pronto para treinar LoRA focada em identidade facial")
print(f"🎭 Trigger word que será treinada: '{INSTANCE_PROMPT}'")

📥 === UPLOAD DO DATASET DE IDENTIDADE ===
Por favor, faça upload do arquivo: valentina_identity_4lora_dataset_flux.zip
🧬 Dataset gerado pelo valentina_dataset_generator_colab.ipynb
📊 Contém 18 imagens com seeds sequenciais para máxima consistência facial


Saving valentina_identity_4lora_dataset_flux.zip to valentina_identity_4lora_dataset_flux.zip
📦 Extraindo valentina_identity_4lora_dataset_flux.zip...
🖼️ Encontradas 18 imagens de identidade
✅ Processada: valentina_dataset_03.png -> valentina_000.png
✅ Processada: valentina_dataset_06.png -> valentina_001.png
✅ Processada: valentina_dataset_05.png -> valentina_002.png
✅ Processada: valentina_dataset_07.png -> valentina_003.png
✅ Processada: valentina_dataset_18.png -> valentina_004.png
✅ Processada: valentina_dataset_17.png -> valentina_005.png
✅ Processada: valentina_dataset_12.png -> valentina_006.png
✅ Processada: valentina_dataset_14.png -> valentina_007.png
✅ Processada: valentina_dataset_09.png -> valentina_008.png
✅ Processada: valentina_dataset_15.png -> valentina_009.png
✅ Processada: valentina_dataset_13.png -> valentina_010.png
✅ Processada: valentina_dataset_02.png -> valentina_011.png
✅ Processada: valentina_dataset_08.png -> valentina_012.png
✅ Processada: valentina_datas

## Célula 6: Preparação do Modelo Base para Treinamento de Identidade
Carrega o modelo FLUX.1-dev oficial e o prepara para treinamento da LoRA de identidade facial.

In [7]:
import torch
from diffusers import FluxPipeline
import gc
import os

print("🚀 Iniciando preparação do modelo base para identidade...")

# Configurar dtype baseado na GPU
if torch.cuda.is_available():
    if torch.cuda.get_device_capability()[0] >= 8:  # Ampere+
        model_dtype = torch.bfloat16
        print("💎 Usando bfloat16 (GPU Ampere+)")
    else:
        model_dtype = torch.float16
        print("⚡ Usando float16 (GPU older)")
else:
    model_dtype = torch.float32
    print("🐌 Usando float32 (CPU)")

# Para o treinamento, o mais simples é usar o modelo ID diretamente
# O script de treinamento irá carregar o modelo automaticamente
model_path = BASE_MODEL_ID
print(f"🎯 Modelo configurado para treinamento: {model_path}")
print("🧬 Foco: Treinamento de identidade facial pura")

# Verificar disponibilidade do modelo
print(f"✅ Modelo base preparado: {BASE_MODEL_ID}")
print("📝 O script de treinamento carregará o modelo automaticamente")

# Alternativamente, se precisar baixar o modelo localmente:
try:
    print(f"\n📦 Verificando disponibilidade do modelo: {BASE_MODEL_ID}")

    # Teste rápido de carregamento para verificar acesso
    pipeline = FluxPipeline.from_pretrained(
        BASE_MODEL_ID,
        torch_dtype=model_dtype,
        use_safetensors=True,
        low_cpu_mem_usage=True
    )

    print("✅ Modelo acessível e compatível!")

    # Salvar localmente se necessário
    model_save_path = f"{COLAB_MODELS_PATH}/flux_base_for_identity"
    print(f"💾 Salvando modelo base em: {model_save_path}")

    pipeline.save_pretrained(
        model_save_path,
        safe_serialization=True
    )

    # Atualizar caminho para apontar para o modelo local
    model_path = model_save_path
    print(f"🎯 Modelo salvo localmente: {model_path}")

    # Verificar componentes do modelo
    print("\n🔍 Componentes do modelo verificados:")
    if hasattr(pipeline, 'transformer'):
        print("   ✅ Transformer (componente principal)")
    if hasattr(pipeline, 'text_encoder'):
        print("   ✅ Text Encoder")
    if hasattr(pipeline, 'text_encoder_2'):
        print("   ✅ Text Encoder 2")
    if hasattr(pipeline, 'vae'):
        print("   ✅ VAE")

    print(f"\n🎯 Modelo preparado: {model_path}")
    print("🧬 Foco: Treinamento de identidade facial pura (sem LoRAs adicionais)")

except Exception as e:
    print(f"⚠️ Não foi possível baixar o modelo localmente: {e}")
    print("🔄 Usando modelo ID diretamente para o treinamento")
    model_path = BASE_MODEL_ID
    print(f"🎯 Modelo para treinamento: {model_path}")

finally:
    # Limpeza de memória
    if 'pipeline' in locals():
        del pipeline
    gc.collect()
    torch.cuda.empty_cache()
    print("🧹 Memória limpa")

print(f"\n✅ Preparação concluída!")
print(f"🎭 Modelo pronto para treinamento de identidade: {model_path}")
print("🧬 Próximo: Iniciar treinamento da LoRA de identidade facial")

🚀 Iniciando preparação do modelo base para identidade...
💎 Usando bfloat16 (GPU Ampere+)
🎯 Modelo configurado para treinamento: black-forest-labs/FLUX.1-dev
🧬 Foco: Treinamento de identidade facial pura
✅ Modelo base preparado: black-forest-labs/FLUX.1-dev
📝 O script de treinamento carregará o modelo automaticamente

📦 Verificando disponibilidade do modelo: black-forest-labs/FLUX.1-dev


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Modelo acessível e compatível!
💾 Salvando modelo base em: /content/models/flux_base_for_identity
🎯 Modelo salvo localmente: /content/models/flux_base_for_identity

🔍 Componentes do modelo verificados:
   ✅ Transformer (componente principal)
   ✅ Text Encoder
   ✅ Text Encoder 2
   ✅ VAE

🎯 Modelo preparado: /content/models/flux_base_for_identity
🧬 Foco: Treinamento de identidade facial pura (sem LoRAs adicionais)
🧹 Memória limpa

✅ Preparação concluída!
🎭 Modelo pronto para treinamento de identidade: /content/models/flux_base_for_identity
🧬 Próximo: Iniciar treinamento da LoRA de identidade facial


## Célula 7: Treinamento da LoRA de Identidade Facial

In [30]:
# Treinamento simplificado da LoRA de identidade facial
import os, shutil, json
%cd /content/fluxgym-Colab

# Copia o dataset de identidade para a pasta do FluxGym
dataset_dst = "/content/fluxgym-Colab/datasets/valentina"
if os.path.exists(dataset_dst):
    shutil.rmtree(dataset_dst)
shutil.copytree(f"{COLAB_DATASET_PATH}/instance_images", dataset_dst)

# Gera arquivo de configuração para o script de treinamento
config = {
    "pretrained_model_name_or_path": BASE_MODEL_ID,
    "train_data_dir": dataset_dst,
    "resolution": "1024,1024",
    "output_dir": COLAB_OUTPUT_PATH,
    "output_name": "valentina_identity_lora",
    "network_dim": 16,
    "learning_rate": 1e-4,
    "max_train_steps": 1800,
    "train_batch_size": 1,
    "mixed_precision": "fp16",
    "save_model_as": "safetensors"
}
with open("configs/valentina.json", "w") as f:
    json.dump(config, f, indent=2)

!python sd-scripts/train_network.py --config_file configs/valentina.json --xformers


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 1153)

## Célula 8: Processamento dos Resultados de Identidade

In [26]:
# Processamento dos Resultados de Identidade
import os, zipfile
from google.colab import files

lora_path = os.path.join(COLAB_OUTPUT_PATH, "valentina_identity_lora.safetensors")
zip_path = "valentina_identity_lora.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    zf.write(lora_path, os.path.basename(lora_path))
files.download(zip_path)


SyntaxError: invalid syntax (<ipython-input-26-c74ebe0db477>, line 217)

## Célula 9: Limpeza (Opcional)
Descomente para remover arquivos grandes e economizar espaço no Colab se for continuar usando o runtime após o treinamento de identidade.

In [ ]:
# print("Limpando arquivos de treinamento de identidade...")
# !rm -rf {COLAB_MODELS_PATH}/{BASE_MODEL_ID.split('/')[-1]} # Remove o cache do modelo base
# !rm -f {DATASET_ZIP_FILENAME} # Remove o dataset ZIP
# !rm -rf {COLAB_DATASET_PATH}/* # Limpa imagens extraídas do dataset de identidade
# print("Limpeza concluída (arquivos de modelo e dataset de identidade).")
# torch.cuda.empty_cache()